In [0]:
import tensorflow as tf

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data', one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [0]:
# 데이터를 넣을 공간인 placeholder 구성하기.
# 데이터 개수는 값을 정해주지 않고 열어두는 의미로 None
# conv2d는 2d 데이터를 받으므로 2d 형태로 28*28,
# 흑백 이미지이므로 channel = 1

X = tf.placeholder(tf.float32, [None, 28, 28, 1])

# 데이터 개수는 오픈해두는 의미로 None
# label 즉, target값이 0~9까지 숫자 이므로 10개의 클래스
Y = tf.placeholder(tf.float32, [None, 10])

# dropout용 placeholder
keep_prob = tf.placeholder(tf.float32)

In [0]:
# 첫 번째 레이어의 필터를 랜덤 정규화(표준편차 0.01)한 값으로 넣기
# 필터 : 5*5 사이즈, input이 흑백인 채널이므로 채널수는 1, 20개의 필터를 만들겠다.
W1 = tf.Variable(tf.random_normal([5, 5, 1, 20], stddev=0.01))

# convolution layer 1 만들기
# layer1은 입력으로 X를 받고 필터 W1을 사용하고
# stride = [샘플수, 세로 길이, 가로 길이, 채널 수]
# padding='SAME' : 추가 패딩은 넣지 않음(사이즈 그대로)
L1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME')

# layer1의 마지막에 활성화 함수 적용
# 활성화 함수는 relu 사용
# 비선형화된 값을 선형화 해주기 위함
L1 = tf.nn.relu(L1)

In [0]:
print(W1)
print(L1)

<tf.Variable 'Variable:0' shape=(5, 5, 1, 20) dtype=float32_ref>
Tensor("Relu:0", shape=(?, 28, 28, 20), dtype=float32)


In [0]:
# 첫 번째 convolution layer 에 Pooling Layer(크기를 줄여주는 레이어) 적용하기
# 입력 :이전 Layer
# ksize = [샘플 수, 세로 사이즈, 가로 사이즈, 채널 수]
# :pooling layer에서의 필터의 사이즈를 의미
# strides = [샘플 수, 세로 스트라이드, 가로 스트라이드, 채널 수]
# :polling layer에서의 필터의 움직임을 의미
# strides 는 가로 세로 각각 1로 하자.
# padding = 'SAME' : 사이즈 유지

L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
print(L1)

Tensor("MaxPool:0", shape=(?, 14, 14, 20), dtype=float32)


In [0]:
# 두 번째 레이어 구성
# 53개의 5*5필터를 만들겠다.
# 필터를 랜덤 정규화(표준편차 0.01)한 값으로 넣기

W2 = tf.Variable(tf.random_normal([5, 5, 20, 53], stddev=0.01))

# conv2d 레이어에서 strides는 가로1, 세로1로 주자.
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')

# activation_function으로 relu 사용
L2 = tf.nn.relu(L2)

# Pooling layer 적용(max_pool)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding='SAME')
print(L2)

Tensor("MaxPool_1:0", shape=(?, 7, 7, 53), dtype=float32)


In [0]:
# 세 번째 레이어(Fully Connected Layer)
# reshape와 행렬곱을 적용함
# 20개의 필터를 만들겠다.
# 필터를 랜덤 정규화(표준편차 0.01)한 값으로 넣기
# 1차원으로 펴주는 flatten 작용이 들어가므로 [4, 4, 53, 20]으로 들어가지 않는다.
# 앞단 필터의 가중치의 갯수, 현재 적용하고자하는 필터의 갯수

W3 = tf.Variable(tf.random_normal([7*7*53, 20], stddev=0.01))

# reshape 과정이 들어간다.
# -1은 몇개의 데이터가 들어올지 모르므로 열어두는 역할
# 뒤에 값은 앞단 필터의 가중치의 갯수

L3 = tf.reshape(L2, [-1, 7*7*53])

# L3와 W3를 행렬곱 해준다.
L3 = tf.matmul(L3, W3)

# 엑티베이션 펑션으로 relu 사용
L3 = tf.nn.relu(L3)

# dropout 을 통해 과적합을 줄여주자.
L3 = tf.nn.dropout(L3, 0.3)

print(L3)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Tensor("dropout/mul_1:0", shape=(?, 20), dtype=float32)


In [0]:
# 마지막 레이어(Flatten Layer)
# 필터를 랜덤 정규화(표준편차 0.01)한 값으로 넣기
# 앞단 필터의 갯수, 현재 적용하고자 하는 필터수를 리스트로 넣어준다.
# 현재 얻고자 하는 필터수는 10개의 숫자 값이므로 10

W4 = tf.Variable(tf.random_normal([20, 10], stddev=0.01))
model = tf.matmul(L3, W4)

In [0]:
# loss 펑션 적용 -> 활성화 함수 : softmax
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels = Y))

# 최적화 방법 : AdamOptimizer 사용
# learning_rate = 0.001
# loss 가 최소가 되는 방향으로 최적화
optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)


In [0]:
# 모델 학습시키기
# session 객체 만들기
sess = tf.Session()

# 변수 초기화를 위한 init 객체 생성
init = tf.global_variables_initializer()

# session 실행
sess.run(init)

In [0]:
# batch_size : 몇개 데이터 당 가중치를 갱신할 것인가
# batch_size가 1이면 데이터 1개를 학습하고 가중치 초기화 ....
# batch_size가 100이면 100개 데이터 학습하고 가중치 초기화
# batch_size를 500으로 설정해보자.

batch_size=500

In [0]:
# total_batch는 데이터 총 량당 가중치 초기화 횟수가 될 것이다.
total_batch = int(mnist.train.num_examples / batch_size)

In [0]:
# epoch : 전체 데이터를 몇번 학습시킬 것인가
# epoch를 3회로 해보자.

for epoch in range(4):
  total_loss = 0
  
  #가중치 초기화할 횟수만큼 돌린다.
  for i in range(batch_size):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    #데이터의 형태를 바꾸어준다.
    batch_xs = batch_xs.reshape(-1, 28, 28, 1)
    
    # X, Y, keep_prob(dropout값, 0.7, 즉 70%만 쓰겠다고 하자)을 feed_dict 값으로 넣어주고,
    # optimizer에 loss값을 넣어 session을 실행한다.
    # 실행 후 값을 반환 받을 때 loss_val만 받는다.
    
    _, loss_val = sess.run([optimizer, loss],
                          feed_dict = {X: batch_xs, Y: batch_ys, keep_prob:0.7})
    
    
    # loss의 평균을 내보기 위해 total_loss에 loss_val값을 더해준다.
    total_loss += loss_val
    
    # Epoch과 loss의 평균을 찍어본다.
  print('Epoch:', '%04d' % (epoch+1),
         'Avg. loss = ', '{:.3f}'.format(total_loss/total_batch))
    
print('최적화 완료!')

Epoch: 0001 Avg. loss =  6.416
Epoch: 0002 Avg. loss =  5.258
Epoch: 0003 Avg. loss =  4.991
Epoch: 0004 Avg. loss =  4.847
최적화 완료!
